In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

import csv
import glob
import logging
import os
import re

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

db_file = 'data/sound_depression.db'
mask_file = '/home/eli/Documents/sound_depression/ds000171-download/all_tasks/wholebrain.nii.gz'

OVERWRITE = True

In [4]:
MASK_FILENAME_TEMPLATE = 'sub-%s%02d_task-%s_run-%02d_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz'

In [5]:
fmriprep_dir = '/home/eli/Documents/sound_depression/fmriprep/'

def mask_filename(condition, subject, task, run):
    return (fmriprep_dir + ('sub-%s%02d' % (condition, subject)) + MASK_FILENAME_TEMPLATE) % (condition, subject, task, run)

In [6]:
existed = os.path.isfile(db_file)
db = niidb.FMriActivationsDb(db_file, mask=mask_file, smooth=6)

if not existed or OVERWRITE:
    data_files = glob.glob('/home/eli/Documents/sound_depression/fmriprep/sub-*/func/*_preproc.nii.gz')
    regex = re.compile("sub-([A-Za-z]+)([0-9]{1,2})_task-([A-Za-z]+)_run-([0-9]{1,2})_\S+preproc.nii.gz")
    for data_file in data_files:
        logging.info("Recording blocks in %s", data_file)
        m = regex.search(data_file)
        condition, subject, task_set, run = m.groups()
        subject = int(subject)
        run = int(run)
        if run is None:
            run = ''
        tsv_file = '/sub-%s%02d_task-%s_run-%01d_events.tsv' % (condition, subject, task_set, run)
        tsv_path = '/home/eli/Documents/sound_depression/ds000171-download/sub-%s%02d/func/' % (condition, subject)
        tsv_file = os.path.dirname(tsv_path) + tsv_file
        with open(tsv_file, 'r') as tsv:
            tsv_reader = csv.DictReader(tsv, dialect='excel-tab')
            for row in tsv_reader:
                onset, duration, task = row.values()
                onset = round(float(onset))
                duration = round(float(duration))
                task_block = niidb.FMriActivationBlock(zscore=True)
                task_block.task = task
                task_block.filename = data_file
                task_block.mask = mask_filename(condition, subject, task, run)
                task_block.subject = subject
                task_block.start_time = onset
                task_block.end_time = onset + duration
                task_block.individual_differences = {'condition': condition}
                db.upsert(task_block)
                logging.info('Stimulus %s from %d to %d', task, onset, onset + duration)

01/11/2019 16:24:54 Recording blocks in /home/eli/Documents/sound_depression/fmriprep/sub-mdd07/func/sub-mdd07_task-music_run-2_bold_space-MNI152NLin2009cAsym_preproc.nii.gz
01/11/2019 16:24:54 Stimulus tones from 0 to 33
01/11/2019 16:24:54 Stimulus response from 33 to 36
01/11/2019 16:24:54 Stimulus positive_music from 36 to 68
01/11/2019 16:24:54 Stimulus response from 68 to 71
01/11/2019 16:24:54 Stimulus tones from 70 to 102
01/11/2019 16:24:54 Stimulus response from 102 to 105
01/11/2019 16:24:54 Stimulus negative_music from 105 to 137
01/11/2019 16:24:54 Stimulus response from 136 to 139
01/11/2019 16:24:54 Stimulus tones from 140 to 172
01/11/2019 16:24:54 Stimulus response from 171 to 174
01/11/2019 16:24:55 Stimulus positive_music from 174 to 206
01/11/2019 16:24:55 Stimulus response from 206 to 209
01/11/2019 16:24:55 Stimulus tones from 208 to 240
01/11/2019 16:24:55 Stimulus response from 240 to 243
01/11/2019 16:24:55 Stimulus negative_music from 243 to 275
01/11/2019 16:

In [7]:
logging.info('Finished building NiiDb out of Lepping 2017 dataset')

01/11/2019 16:27:36 Finished building NiiDb out of Lepping 2017 dataset
